In [12]:
from math import *
import matplotlib.pyplot as plt
import matplotlib as mpl
import subprocess as sp
import os
import glob
import re
import gzip
import numpy as np
import pickle
from Photochem import *
from Climate import *
from Constants import *
from Technical_functions import *

plt.style.use('default')
mpl.rcParams['figure.dpi'] = 1200
mpl.rcParams['axes.facecolor'] = (1,1,1,0)
plt.ioff()

In [13]:
## Loads the outputs of the 203K freezing point scenario
k = 0
job_dir = 'out_rho_203/'

# Lists the results files and corresponding parameter files
results_list_203 = sorted(glob.glob(job_dir+'*.results_dc'))
params_list_203  = sorted(glob.glob(job_dir+'*.params'))

# Checks whether all parameter files have result files attached to it
to_be_kept = []
for i in np.arange(len(params_list_203)):
    text  = params_list_203[i]
    index = re.search('job.(.+?)params', text).group(1)    
    check = 0
    for text in results_list_203:
        if index in text:
            check += 1
    if check == 1:
        to_be_kept += [i]
    
# Update the parameter list accordingly
params_list_203 = [params_list_203[i] for i in to_be_kept]

# Prepare the storage matrices
a_eps_vec_203   = []        # Near-surface porosity 
a_to_vec_203    = []        # Near-surface tortuosity
r_surf_vec_203  = []        # Near-surface pore radius (cm) 
ztot_vec_203    = []        # Pore closure depth (km)
NCT_mat_203     = []        # Cell density depth profile (cell.L-1)
z_mat_203       = []        # Depth layers
a_T_vec_203     = []        # Underground temperature gradient (K.km-1)
Volc_vec_203    = []        # Volcanic H2 outgassing (molecule.s-1.cm-2)
T_surfe_vec_203 = []        # Steady state average temperature in the habitable region of Mars
T_ave_vec_203   = []        # Steady state average temperature on Mars
rhoe_vec_203    = []        # Steady state habitable portion of Mars 
T_surf_vec_203  = []        # Initial average temperature in the habitable region of Mars
T_av_vec_203    = []        # Initial average temperature on Mars
rho_vec_203     = []        # Initial habitable portion of Mars 
pH2_vec_203     = []        # Initial atm. H2 (proportion)
pCH4_vec_203    = []        # Initial atm. CH4 (proportion)
pCO2_vec_203    = []        # Initial atm. CO2 (proportion)
p_vec_203       = []        # Initial atmospheric pressure (Pascals)
DiffH_vec_203   = []        # Initial crust-Atm. H2 flux (molecule.s-1.cm-2)
DiffG_vec_203   = []        # Initial crust-Atm. CH4 flux (molecule.s-1.cm-2)
NPP_vec_203     = []        # Initial Biomass productivity (molecule.s-1.cm-2)
pH2e_vec_203    = []        # Steady state atm. H2 (proportion)
pCH4e_vec_203   = []        # Steady state atm. CH4 (proportion)
pe_vec_203      = []        # Steady state atmospheric pressure (Pascals)
DiffHe_vec_203  = []        # Steady state crust-Atm. H2 flux (molecule.s-1.cm-2)
DiffGe_vec_203  = []        # Steady state crust-Atm. CH4 flux (molecule.s-1.cm-2)
NPPe_vec_203    = []        # Steady state Biomass productivity (molecule.s-1.cm-2)
fail_203        = []        # Just to keep track of when the results/parameter files are corrupted

# Go through all the files
k=0
for i in np.arange(k,len(results_list_203)):

    # Open a specific simulation's files
    # Side note: these 4 lines of code are how to open the model outputs
    f = gzip.open(results_list_203[i],'rb')
    t,y,pT,T_surfT,T_avT,rhoT,pCH4T,pH2T,pCO2T,pN2T,DiffHT,DiffGT = pickle.load(f)
    p,pH2,pCO2,pCH4,a_eps,a_to,r_surf,T_av,ztot,a_T = pickle.load(open(params_list_203[i],'rb'))
    f.close()
    
    if (t[-1] > 1 and T_surfT[-1] > T_avT[-1]) or rhoT[0] == 0: # Check whether there has been a fail
        # Saves the results
        layers = 100
        z_vec = np.linspace(0,ztot,layers)

        HT       = np.transpose(y)[(0*layers):(0*layers+layers)]
        CT       = np.transpose(y)[(1*layers):(1*layers+layers)]
        NT       = np.transpose(y)[(3*layers):(3*layers+layers)]
        GT       = np.transpose(y)[(2*layers):(2*layers+layers)]
        NCT      = np.transpose(y)[(4*layers):(4*layers+layers)]
        X0T      = np.transpose(y)[(5*layers):(5*layers+layers)]

        #pCH4     = GEnv_t[-1]/(p*1e-5*alphaG*1e-3*Av)
        pCH4            = pCH4T[-1]
        E               = 1.6e13          
        Esc             = E*(pH2+pCH4/2)
        F_CH4           = float(FCH4_func(pH2,pCH4))
        F_H2            = float(FH2_func(pH2,pCH4))
        Volc            = Esc - F_H2

        NCT_mat_203     += [np.transpose(NCT)[-1].tolist()]
        z_mat_203       += [z_vec.tolist()]
        T_surfe_vec_203 += [T_surfT[-1]]
        T_ave_vec_203   += [T_avT[-1]]
        rhoe_vec_203    += [rhoT[-1]]
        T_surf_vec_203  += [T_surfT[0]]
        T_av_vec_203    += [T_avT[0]]
        rho_vec_203     += [rhoT[0]]
        pH2_vec_203     += [pH2]
        pCH4_vec_203    += [pCH4T[0]]
        pH2e_vec_203    += [pH2T[-1]]
        pCH4e_vec_203   += [pCH4T[-1]]
        pCO2_vec_203    += [pCO2]
        p_vec_203       += [p]
        pe_vec_203      += [pT[-1]]    
        a_eps_vec_203   += [a_eps]
        a_to_vec_203    += [a_to]
        Volc_vec_203    += [Volc]
        r_surf_vec_203  += [r_surf]
        ztot_vec_203    += [ztot]
        a_T_vec_203     += [a_T]
        DiffH_vec_203   += [DiffHT[1]]
        DiffG_vec_203   += [DiffGT[1]]
        DiffHe_vec_203  += [DiffHT[-1]]
        DiffGe_vec_203  += [DiffGT[-1]]
        NPPT_vec_203     = [sum(np.transpose(X0T)[i]*Av*np.transpose(NCT)[i]*np.mean(diff(z_vec))*1e5)/(24*60*60)*0.1*1e-3 for i in range(len(t))]
        NPP_vec_203     += [np.max(NPPT_vec_203)]
        NPPe_vec_203    += [sum(np.transpose(X0T)[-1]*Av*np.transpose(NCT)[-1]*np.mean(diff(z_vec))*1e5)/(24*60*60)*0.1*1e-3]
    else:
        fail_203        += [k]
    print(k,end='\r')
    k+=1

    
## Loads the outputs of the 252K freezing point scenario
k = 0
job_dir = 'out_rho_252/'

# Lists the results files and corresponding parameter files
results_list_252 = sorted(glob.glob(job_dir+'*.results_dc'))
params_list_252  = sorted(glob.glob(job_dir+'*.params'))

# Checks whether all parameter files have result files attached to it
to_be_kept = []
for i in np.arange(len(params_list_252)):
    text  = params_list_252[i]
    index = re.search('job.(.+?)params', text).group(1)    
    check = 0
    for text in results_list_252:
        if index in text:
            check += 1
    if check == 1:
        to_be_kept += [i]
    
# Update the parameter list accordingly
params_list_252 = [params_list_252[i] for i in to_be_kept]

# Prepare the storage matrices
a_eps_vec_252   = []        # Near-surface porosity 
a_to_vec_252    = []        # Near-surface tortuosity
r_surf_vec_252  = []        # Near-surface pore radius (cm) 
ztot_vec_252    = []        # Pore closure depth (km)
NCT_mat_252     = []        # Cell density depth profile (cell.L-1)
z_mat_252       = []        # Depth layers
a_T_vec_252     = []        # Underground temperature gradient (K.km-1)
Volc_vec_252    = []        # Volcanic H2 outgassing (molecule.s-1.cm-2)
T_surfe_vec_252 = []        # Steady state average temperature in the habitable region of Mars
T_ave_vec_252   = []        # Steady state average temperature on Mars
rhoe_vec_252    = []        # Steady state habitable portion of Mars 
T_surf_vec_252  = []        # Initial average temperature in the habitable region of Mars
T_av_vec_252    = []        # Initial average temperature on Mars
rho_vec_252     = []        # Initial habitable portion of Mars 
pH2_vec_252     = []        # Initial atm. H2 (proportion)
pCH4_vec_252    = []        # Initial atm. CH4 (proportion)
pCO2_vec_252    = []        # Initial atm. CO2 (proportion)
p_vec_252       = []        # Initial atmospheric pressure (Pascals)
DiffH_vec_252   = []        # Initial crust-Atm. H2 flux (molecule.s-1.cm-2)
DiffG_vec_252   = []        # Initial crust-Atm. CH4 flux (molecule.s-1.cm-2)
NPP_vec_252     = []        # Initial Biomass productivity (molecule.s-1.cm-2)
pH2e_vec_252    = []        # Steady state atm. H2 (proportion)
pCH4e_vec_252   = []        # Steady state atm. CH4 (proportion)
pe_vec_252      = []        # Steady state atmospheric pressure (Pascals)
DiffHe_vec_252  = []        # Steady state crust-Atm. H2 flux (molecule.s-1.cm-2)
DiffGe_vec_252  = []        # Steady state crust-Atm. CH4 flux (molecule.s-1.cm-2)
NPPe_vec_252    = []        # Steady state Biomass productivity (molecule.s-1.cm-2)
fail_252        = []        # Just to keep track of when the results/parameter files are corrupted

# Go through all the files
k=0
for i in np.arange(k,len(results_list_252)):

    # Open a specific simulation's files
    # Side note: these 4 lines of code are how to open the model outputs
    f = gzip.open(results_list_252[i],'rb')
    t,y,pT,T_surfT,T_avT,rhoT,pCH4T,pH2T,pCO2T,pN2T,DiffHT,DiffGT = pickle.load(f)
    p,pH2,pCO2,pCH4,a_eps,a_to,r_surf,T_av,ztot,a_T = pickle.load(open(params_list_252[i],'rb'))
    f.close()
    
    if (t[-1] > 1 and T_surfT[-1] > T_avT[-1]) or rhoT[0] == 0: # Check whether there has been a fail
        # Saves the results
        layers = 100
        z_vec = np.linspace(0,ztot,layers)

        HT       = np.transpose(y)[(0*layers):(0*layers+layers)]
        CT       = np.transpose(y)[(1*layers):(1*layers+layers)]
        NT       = np.transpose(y)[(3*layers):(3*layers+layers)]
        GT       = np.transpose(y)[(2*layers):(2*layers+layers)]
        NCT      = np.transpose(y)[(4*layers):(4*layers+layers)]
        X0T      = np.transpose(y)[(5*layers):(5*layers+layers)]

        #pCH4     = GEnv_t[-1]/(p*1e-5*alphaG*1e-3*Av)
        pCH4            = pCH4T[-1]
        E               = 1.6e13          
        Esc             = E*(pH2+pCH4/2)
        F_CH4           = float(FCH4_func(pH2,pCH4))
        F_H2            = float(FH2_func(pH2,pCH4))
        Volc            = Esc - F_H2

        NCT_mat_252     += [np.transpose(NCT)[-1].tolist()]
        z_mat_252       += [z_vec.tolist()]
        T_surfe_vec_252 += [T_surfT[-1]]
        T_ave_vec_252   += [T_avT[-1]]
        rhoe_vec_252    += [rhoT[-1]]
        T_surf_vec_252  += [T_surfT[0]]
        T_av_vec_252    += [T_avT[0]]
        rho_vec_252     += [rhoT[0]]
        pH2_vec_252     += [pH2]
        pCH4_vec_252    += [pCH4T[0]]
        pH2e_vec_252    += [pH2T[-1]]
        pCH4e_vec_252   += [pCH4T[-1]]
        pCO2_vec_252    += [pCO2]
        p_vec_252       += [p]
        pe_vec_252      += [pT[-1]]    
        a_eps_vec_252   += [a_eps]
        a_to_vec_252    += [a_to]
        Volc_vec_252    += [Volc]
        r_surf_vec_252  += [r_surf]
        ztot_vec_252    += [ztot]
        a_T_vec_252     += [a_T]
        DiffH_vec_252   += [DiffHT[1]]
        DiffG_vec_252   += [DiffGT[1]]
        DiffHe_vec_252  += [DiffHT[-1]]
        DiffGe_vec_252  += [DiffGT[-1]]
        NPPT_vec_252     = [sum(np.transpose(X0T)[i]*Av*np.transpose(NCT)[i]*np.mean(diff(z_vec))*1e5)/(24*60*60)*0.1*1e-3 for i in range(len(t))]
        NPP_vec_252     += [np.max(NPPT_vec_252)]
        NPPe_vec_252    += [sum(np.transpose(X0T)[-1]*Av*np.transpose(NCT)[-1]*np.mean(diff(z_vec))*1e5)/(24*60*60)*0.1*1e-3]
    else:
        fail_252        += [k]
    print(k,end='\r')
    k+=1

    
## Loads the outputs of the 273K freezing point scenario
k = 0
job_dir = 'out_rho_273/'

# Lists the results files and corresponding parameter files
results_list_273 = sorted(glob.glob(job_dir+'*.results_dc'))
params_list_273  = sorted(glob.glob(job_dir+'*.params'))

# Checks whether all parameter files have result files attached to it
to_be_kept = []
for i in np.arange(len(params_list_273)):
    text  = params_list_273[i]
    index = re.search('job.(.+?)params', text).group(1)    
    check = 0
    for text in results_list_273:
        if index in text:
            check += 1
    if check == 1:
        to_be_kept += [i]
    
# Update the parameter list accordingly
params_list_273 = [params_list_273[i] for i in to_be_kept]

# Prepare the storage matrices
a_eps_vec_273   = []        # Near-surface porosity 
a_to_vec_273    = []        # Near-surface tortuosity
r_surf_vec_273  = []        # Near-surface pore radius (cm) 
ztot_vec_273    = []        # Pore closure depth (km)
NCT_mat_273     = []        # Cell density depth profile (cell.L-1)
z_mat_273       = []        # Depth layers
a_T_vec_273     = []        # Underground temperature gradient (K.km-1)
Volc_vec_273    = []        # Volcanic H2 outgassing (molecule.s-1.cm-2)
T_surfe_vec_273 = []        # Steady state average temperature in the habitable region of Mars
T_ave_vec_273   = []        # Steady state average temperature on Mars
rhoe_vec_273    = []        # Steady state habitable portion of Mars 
T_surf_vec_273  = []        # Initial average temperature in the habitable region of Mars
T_av_vec_273    = []        # Initial average temperature on Mars
rho_vec_273     = []        # Initial habitable portion of Mars 
pH2_vec_273     = []        # Initial atm. H2 (proportion)
pCH4_vec_273    = []        # Initial atm. CH4 (proportion)
pCO2_vec_273    = []        # Initial atm. CO2 (proportion)
p_vec_273       = []        # Initial atmospheric pressure (Pascals)
DiffH_vec_273   = []        # Initial crust-Atm. H2 flux (molecule.s-1.cm-2)
DiffG_vec_273   = []        # Initial crust-Atm. CH4 flux (molecule.s-1.cm-2)
NPP_vec_273     = []        # Initial Biomass productivity (molecule.s-1.cm-2)
pH2e_vec_273    = []        # Steady state atm. H2 (proportion)
pCH4e_vec_273   = []        # Steady state atm. CH4 (proportion)
pe_vec_273      = []        # Steady state atmospheric pressure (Pascals)
DiffHe_vec_273  = []        # Steady state crust-Atm. H2 flux (molecule.s-1.cm-2)
DiffGe_vec_273  = []        # Steady state crust-Atm. CH4 flux (molecule.s-1.cm-2)
NPPe_vec_273    = []        # Steady state Biomass productivity (molecule.s-1.cm-2)
fail_273        = []        # Just to keep track of when the results/parameter files are corrupted

# Go through all the files
k=0
for i in np.arange(k,len(results_list_273)):

    # Open a specific simulation's files
    # Side note: these 4 lines of code are how to open the model outputs
    f = gzip.open(results_list_273[i],'rb')
    t,y,pT,T_surfT,T_avT,rhoT,pCH4T,pH2T,pCO2T,pN2T,DiffHT,DiffGT = pickle.load(f)
    p,pH2,pCO2,pCH4,a_eps,a_to,r_surf,T_av,ztot,a_T = pickle.load(open(params_list_273[i],'rb'))
    f.close()
    
    if (t[-1] > 1 and T_surfT[-1] > T_avT[-1]) or rhoT[0] == 0: # Check whether there has been a fail
        # Saves the results
        layers = 100
        z_vec = np.linspace(0,ztot,layers)

        HT       = np.transpose(y)[(0*layers):(0*layers+layers)]
        CT       = np.transpose(y)[(1*layers):(1*layers+layers)]
        NT       = np.transpose(y)[(3*layers):(3*layers+layers)]
        GT       = np.transpose(y)[(2*layers):(2*layers+layers)]
        NCT      = np.transpose(y)[(4*layers):(4*layers+layers)]
        X0T      = np.transpose(y)[(5*layers):(5*layers+layers)]

        #pCH4     = GEnv_t[-1]/(p*1e-5*alphaG*1e-3*Av)
        pCH4            = pCH4T[-1]
        E               = 1.6e13          
        Esc             = E*(pH2+pCH4/2)
        F_CH4           = float(FCH4_func(pH2,pCH4))
        F_H2            = float(FH2_func(pH2,pCH4))
        Volc            = Esc - F_H2

        NCT_mat_273     += [np.transpose(NCT)[-1].tolist()]
        z_mat_273       += [z_vec.tolist()]
        T_surfe_vec_273 += [T_surfT[-1]]
        T_ave_vec_273   += [T_avT[-1]]
        rhoe_vec_273    += [rhoT[-1]]
        T_surf_vec_273  += [T_surfT[0]]
        T_av_vec_273    += [T_avT[0]]
        rho_vec_273     += [rhoT[0]]
        pH2_vec_273     += [pH2]
        pCH4_vec_273    += [pCH4T[0]]
        pH2e_vec_273    += [pH2T[-1]]
        pCH4e_vec_273   += [pCH4T[-1]]
        pCO2_vec_273    += [pCO2]
        p_vec_273       += [p]
        pe_vec_273      += [pT[-1]]    
        a_eps_vec_273   += [a_eps]
        a_to_vec_273    += [a_to]
        Volc_vec_273    += [Volc]
        r_surf_vec_273  += [r_surf]
        ztot_vec_273    += [ztot]
        a_T_vec_273     += [a_T]
        DiffH_vec_273   += [DiffHT[1]]
        DiffG_vec_273   += [DiffGT[1]]
        DiffHe_vec_273  += [DiffHT[-1]]
        DiffGe_vec_273  += [DiffGT[-1]]
        NPPT_vec_273     = [sum(np.transpose(X0T)[i]*Av*np.transpose(NCT)[i]*np.mean(diff(z_vec))*1e5)/(24*60*60)*0.1*1e-3 for i in range(len(t))]
        NPP_vec_273     += [np.max(NPPT_vec_273)]
        NPPe_vec_273    += [sum(np.transpose(X0T)[-1]*Av*np.transpose(NCT)[-1]*np.mean(diff(z_vec))*1e5)/(24*60*60)*0.1*1e-3]
    else:
        fail_273        += [k]
    print(k,end='\r')
    k+=1

IndexError: index 1 is out of bounds for axis 0 with size 1